In [16]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
import torch
import numpy as np
import utils
import matplotlib.pyplot as plt
from dataset import DataSet
import pandas as pd
%load_ext autoreload
np.random.seed(2) 
torch.manual_seed(2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
%autoreload

In [12]:
lcs = utils.get_data('../../datasets/ZTF_all', split=1.0)


validated 3407 files out of 3407 for band='r'
validated 3407 files out of 3407 for band='i'
validated 3408 files out of 3408 for band='g'
created union_tp attribute of length 3500
dataset created w/ shape (2683, 3, 4431, 3)
train size: 2683, valid size: 0, test size: 0


In [13]:
net, optimizer, args, epoch, loss, train_loss, test_loss = utils.load_checkpoint('../checkpoints/ZTF_all_noz0.7991786003112793.h5', lcs.data_obj)


=> loading checkpoint '../checkpoints/ZTF_all_noz0.7991786003112793.h5'
Namespace(n_union_tp=3500, data_folder='datasets/ZTF_all_noz', checkpoint='ZTF_all_noz0.7986480593681335.h5', start_col=1, inc_errors=False, print_at=1, embed_time=128, enc_num_heads=16, latent_dim=64, mixing='concat', num_ref_points=16, rec_hidden=128, width=512, save_at=30, patience=100, early_stopping=False, niters=20000, frac=0.5, batch_size=2, mse_weight=5.0, dropout=0.0, num_resamples=0, lr=1e-08, scheduler=False, warmup=4000, kl_zero=False, kl_annealing=True, net='hetvae', device='mps', const_var=False, var_per_dim=False, std=0.1, seed=2, save=True, k_iwae=1)


In [19]:
# make encoding with only first half of recorded points for each light curve

qzs,disc_path = utils.encode(lcs.data_obj['train_loader'], net, device='mps',subsample=True,forecast=True) 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1342/1342 [00:45<00:00, 29.26it/s]


In [32]:
target_x = lcs.dataset[:,:,:,0]

In [33]:
## take samples from qz, decode separately, then average the interpolations 
n_samples = 10
zs = (np.random.randn(n_samples, qzs.shape[0], qzs.shape[2],qzs.shape[3]) * qzs[:,1,:,:] + qzs[:,0,:,:])

In [34]:
interps = np.array([utils.decode(net,z[:10],disc_path[:10],target_x[:10]) for z in zs])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.11s/it]


In [35]:
interps.shape

(10, 10, 3, 4431, 3)

In [ ]:
t = interps[0,:,:,:,0]
y_avg = interps[:,:,:,:,1].mean(0)
yerr_avg = interps[:,:,:,:,2].mean(0)
intps = np.concatenate((t[np.newaxis],y_avg[np.newaxis],yerr_avg[np.newaxis]),axis=0).transpose(1,2,3,0)

### UNNORMALIZE

In [ ]:
for i, object_lcs in enumerate(lcs.unnormalized_data[:10]): 
    min_t = 1000000
    for lc in object_lcs:
        if lc[:,1].any():
            if lc[0,0] < min_t:
                min_t = lc[0,0]
                
    for j, lc in enumerate(object_lcs):
        if lc[:,1].any():
            mean = np.mean(lc[:,1])
            std = np.std(lc[:,1])
            intps[i,j,:,0] = intps[i,j,:,0] + min_t
            intps[i,j,:,1] = (intps[i,j,:,1] * std) + mean  
            intps[i,j,:,2] = (intps[i,j,:,2] * std)


In [ ]:
n=len(interps)
dims = len(lcs.bands)
figsize=(30,30)
fig,ax = plt.subplots(n,dims,figsize=figsize, squeeze=False)
for ex in range(n):
    ax[ex,0].set_ylabel(f'bin {ex}')
    for band in range(dims):
        t=intps[ex,band,:,0]
        y=intps[ex,band,:,1]
        yerr=intps[ex,band,:,2]
        ax[ex,band].fill_between(t,y-yerr,y+yerr, label='error envelope',color='lightcoral')
        ax[ex,band].errorbar(t,y,yerr=yerr, c='blue', ecolor='#65c9f7', label='prediction')
        #ax[ex,band].scatter(t,y)
        if lcs.unnormalized_data[ex][band][:,1].any(): # not missing light curves
            ax[ex,band].scatter(lcs.unnormalized_data[ex][band][:,0], lcs.unnormalized_data[ex][band][:,1])
            

In [ ]:
# setting make masks to first half of the points, for only test set
# take MSE / NLL wrt second half of the points for the test set